In [1]:
import os
import csv
from torchvision.io import read_image
import pandas as pd
from sklearn.model_selection import train_test_split
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch.optim as optim

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class AlexNet(nn.Module):
    def __init__(self, num_classes):
        super(AlexNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 96, kernel_size = 11, stride = 4, padding = 0)
        self.relu1 = nn.ReLU()
        self.response_Norm1 = nn.LocalResponseNorm(size = 5,k = 2)
        self.max_pool1 = nn.MaxPool2d(kernel_size = 3, stride = 2)

        self.conv2 = nn.Conv2d(96, 256, kernel_size = 5,  stride = 1, padding = 'same')
        self.relu2 = nn.ReLU()
        self.response_Norm2 = nn.LocalResponseNorm(size = 5,k = 2)
        self.max_pool2 = nn.MaxPool2d(kernel_size = 3, stride = 2)

        self.conv3 =  nn.Conv2d(256, 384, kernel_size = 3, stride = 1, padding = 'same')
        self.relu3 = nn.ReLU()

        self.conv4 =  nn.Conv2d(384, 384, kernel_size = 3, stride = 1, padding = 'same')
        self.relu4 = nn.ReLU()

        self.conv5 =  nn.Conv2d(384, 256, kernel_size = 3, stride = 1, padding = 'same')
        self.relu5 = nn.ReLU()
        self.response_Norm3 = nn.LocalResponseNorm(size = 5,k = 2)
        self.max_pool3 = nn.MaxPool2d(kernel_size = 3, stride = 2)

        self.dropout1 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(256 * 6 * 6, 4096)
        self.relu1 = nn.ReLU()

        self.dropout2 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(4096, 4096)
        self.relu2 = nn.ReLU()

        self.fc3 = nn.Linear(4096,num_classes)
    
    
    def forward(self,x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.response_Norm1(x)
        x = self.max_pool1(x)

        x = self.conv2(x)
        x = self.relu2(x)
        x = self.response_Norm2(x)
        x = self.max_pool2(x)

        x = self.conv3(x)
        x = self.relu3(x)
        x = self.conv4(x)
        x = self.relu4(x)

        x = self.conv5(x)
        x = self.relu5(x)
        x = self.response_Norm3(x)
        x = self.max_pool3(x)

        x = torch.flatten(x, start_dim=1)

        x = self.dropout1(x)
        x = self.fc1(x)
        x = self.relu1(x)

        x = self.dropout2(x)
        x = self.fc2(x)
        x = self.relu2(x)
        
        x = self.fc3(x)
        return x

In [3]:
model = AlexNet(5)

In [4]:
# Define data transformations
transform = transforms.Compose([
    transforms.Resize((227, 227)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load custom dataset
custom_dataset = ImageFolder(root='../data/images/', transform=transform)

# Split the dataset
train_size = int(0.8 * len(custom_dataset))
val_size = len(custom_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(custom_dataset, [train_size, val_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4)


In [5]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}] - "
          f"Validation Loss: {val_loss/len(val_loader):.4f}, "
          f"Validation Accuracy: {(100*correct/total):.2f}%")

RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x5 and 4096x5)